<a href="https://colab.research.google.com/github/DunkleCat/ia-esame/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!pip install -r requirements.txt

# Caricamento del dataframe

In [56]:
import pandas as pd

def io_load_multiple_csv(csv_path_list):
  dataframe_list = []
  for elem in csv_path_list:
    dataframe_list.append(io_load_csv(elem))
  return dataframe_list

def io_load_csv(csv_path):
  return pd.read_csv(csv_path)  

# Analisi del Dataset


## Normalizzazione valori nulli

Come primo passaggio normalizziamo i valori che rappresentano attributi mancanti trasformandoli tutti in np.nan


In [57]:
import numpy as np

def normalize_nan(dataframe, nan_list):
  for elem in nan_list:
    dataframe.replace(elem, np.nan, inplace=True)
  return dataframe

## Analisi dataframe

Passiamo ora ad analizzare i dati in ingresso

In [58]:
def print_infos(dataframe):
  print("Dataframe miscellaneous:\n")
  print("Rows     : {}".format(dataframe.shape[0]) )
  print("Columns  : {}".format(dataframe.shape[1]))
  print("\nFeatures :\n{}".format(dataframe.columns.tolist()))
  print("\nUnique values :\n{}".format(dataframe.nunique()))

  print("\nDataframe info:")
  dataframe.info()

In [59]:
import matplotlib.pyplot as plt
import seaborn as sb

def print_feature_plots(dataframe, feature_list, feature_target):
  for elem in dataframe:
    if elem in feature_list:
      plot = sb.catplot(x = feature_target, 
                        col = elem, 
                        data = dataframe, 
                        kind = 'count')
    plt.show()

Questa funzione, data una lista di feature, elimina dal dataframe in input tutte le colonne appartenti alla lista precedente. 

In [60]:
def clean_useless(dataframe, column_list):
  for elem in column_list:
    dataframe.pop(elem)
  return dataframe

## Altre informazioni temporali

In [61]:
from datetime import date

def add_diffence_scheduled_appointment_day(dataframe):
  
  appointmentWeekdayDay = []
  daysFromScheduledToAppointment = []

  for i in range(dataframe['AppointmentDay'].size):
    appointmentWeekdayDay.append(get_weekday(dataframe['AppointmentDay'][i]))
    daysFromScheduledToAppointment.append(get_diff_days(dataframe['ScheduledDay'][i], dataframe['AppointmentDay'][i]))
  
  dataframe['AppointmentWeekDay'] = appointmentWeekdayDay
  dataframe['DaysFromScheduledToAppointment'] = daysFromScheduledToAppointment
  return dataframe

# string_date = YYYY-MM-DD[.*]
def get_date(string_date):
  return date(year = int(string_date[0:4]), 
              month = int(string_date[5:7]), 
              day = int(string_date[8:10]))

def get_weekday(string_date):
  return get_date(string_date).weekday()

def get_diff_days(string_date_start, string_date_end): 
  return (get_date(string_date_end) - get_date(string_date_start)).days


# Modello

Vengono sfruttate le possiilità offerte da una pipeline di sklearn per creare ed utilizzare il modello.

Come classificatore viene utilizzato l'algoritmo di Random Forest. Esso è un algoritmo di tipo Ensemble, e quindi sfrutta la combinazione di altri algoritmi più deboli ma opportunamente organizzati ed utilizzati per ottenere dei risultati soddisfacenti. Nel caso specifico abbiamo una combinazione di classificatori di tipo Decision tree.

In [62]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

def create_model(dataframe):

  preprocessor = create_preprocessor(dataframe)

  return Pipeline(
      steps = [('preprocessor', preprocessor),
               ('classifier', RandomForestClassifier())])

def create_preprocessor(dataframe):
  numeric_features = get_numeric_features(dataframe)
  numeric_transformer = Pipeline(
      steps = [('imputer', SimpleImputer(strategy='median')),
               ('scaler', StandardScaler())])

  categorical_features = get_categorical_features(dataframe)
  categorical_transformer = Pipeline(
      steps=[('imputer', SimpleImputer(strategy='most_frequent')),
             ('onehot', OneHotEncoder(handle_unknown='ignore'))])

  return ColumnTransformer(
      transformers=[('num', numeric_transformer, numeric_features),
                    ('cat', categorical_transformer, categorical_features)])

def get_numeric_features(dataframe):
  numeric_features = []
  for elem in dataframe:
    if is_numeric(dataframe[elem]):
      numeric_features.append(elem)
  return numeric_features

def get_categorical_features(dataframe):
  categorical_features = []
  for elem in dataframe:
    if is_categorical(dataframe[elem]):
      categorical_features.append(elem)
  return categorical_features

def is_numeric(elem):
  return elem is float or elem is int

def is_categorical(elem):
  return not is_numeric(elem)

In [63]:
from sklearn import set_config

def print_model(model):
  #set_config(display='diagram')
  model

In [64]:
from sklearn.model_selection import GridSearchCV

def train_model(model, dataframe_train, dataframe_train_target):
  return model.fit(dataframe_train, dataframe_train_target)

In [65]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, plot_confusion_matrix

def print_test_model(model, dataframe_test, dataframe_test_target):
  preds = model.predict(dataframe_test)
  print('Test score: {}'.format(model.score(dataframe_test, dataframe_test_target)))
 
  print(classification_report(dataframe_test_target, preds))

# Esecuzione

In [66]:
## MISCELLANEOUS ###############################################################

# Rende i numeri a virgola mobile più leggibili
np.set_printoptions(precision=3, suppress=True)

################################################################################

In [67]:
## MANUAL INTERVENTION #########################################################

dataframe_train_path = "train.csv"
dataframe_test_path = "test.csv"

################################################################################

## EXECUTION ###################################################################

dataframe_train, dataframe_test = io_load_multiple_csv([dataframe_train_path, 
                                                        dataframe_test_path])

################################################################################

In [68]:
dataframe_train.head()

,No-show,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,No,2.647823e+12,5626550,M,2016-04-27T08:22:02Z,2016-05-05T00:00:00Z,63,ILHA DO PRÍNCIPE,0,1,1,0,0,1
1,No,2.772585e+14,5680940,F,2016-05-10T13:18:27Z,2016-05-13T00:00:00Z,52,SANTO ANTÔNIO,0,0,0,0,0,0
2,No,5.378235e+13,5679296,F,2016-05-10T09:44:35Z,2016-05-10T00:00:00Z,46,CENTRO,0,1,0,0,0,0
3,No,2.711119e+13,5621478,F,2016-04-26T10:21:41Z,2016-05-02T00:00:00Z,31,DA PENHA,0,0,0,0,0,0
4,No,8.328715e+10,5671891,F,2016-05-09T07:22:43Z,2016-05-11T00:00:00Z,50,CONQUISTA,0,0,0,0,0,0


In [69]:
## MANUAL INTERVENTION #########################################################

nan_list = [
            "", 
            " ", 
            "?",
            "unknown"
] # All the ways a nan element appears inside the dataframe

feature_target = "No-show"

################################################################################

## EXECUTION ###################################################################

# Clean the dataframe from nan values
dataframe_train = normalize_nan(dataframe_train, nan_list)
dataframe_test = normalize_nan(dataframe_test, nan_list)

# Print dataframe's infographic
print_infos(dataframe_train)

################################################################################

Dataframe miscellaneous:

Rows     : 70736
Columns  : 14

Features :
['No-show', 'PatientId', 'AppointmentID', 'Gender', 'ScheduledDay', 'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received']

Unique values :
No-show               2
PatientId         46573
AppointmentID     70736
Gender                2
ScheduledDay      67701
AppointmentDay       27
Age                 103
Neighbourhood        81
Scholarship           2
Hipertension          2
Diabetes              2
Alcoholism            2
Handcap               5
SMS_received          2
dtype: int64

Dataframe info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70736 entries, 0 to 70735
Data columns (total 14 columns):
No-show           70736 non-null object
PatientId         70736 non-null float64
AppointmentID     70736 non-null int64
Gender            70736 non-null object
ScheduledDay      70736 non-null object
AppointmentDay    70736 non-null object
Age  

In [70]:
## MANUAL INTERVENTION #########################################################

useless_list = [
                "AppointmentID"       
] # All the meaningless attributes in relation to the prediction 

################################################################################

## EXECUTION ###################################################################

# Clean the dataframe from meningless features
dataframe_train = clean_useless(dataframe_train, useless_list)
dataframe_test = clean_useless(dataframe_test, useless_list)

# Add some useful features
dataframe_train = add_diffence_scheduled_appointment_day(dataframe_train)
dataframe_test = add_diffence_scheduled_appointment_day(dataframe_test)

################################################################################


In [71]:
from sklearn.preprocessing import LabelEncoder

## MANUAL INTERVENTION #########################################################

feature_target = "No-show"

################################################################################

## TARGET PREPARATION ##########################################################

le = LabelEncoder()

dataframe_train_target = le.fit_transform(dataframe_train.pop(feature_target))
dataframe_test_target = le.transform(dataframe_test.pop(feature_target))

################################################################################

In [72]:
## CLASSIFIER ##################################################################

# Create the classifier
classifier = create_model(dataframe_train)
print_model(classifier)

# Train the classifier
classifier = train_model(classifier, dataframe_train, dataframe_train_target)

# Test the classifier
print_test_model(classifier, dataframe_test, dataframe_test_target)

################################################################################

Test score: 0.8048063330506079
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     14119
           1       0.82      0.04      0.08      3566

    accuracy                           0.80     17685
   macro avg       0.81      0.52      0.48     17685
weighted avg       0.81      0.80      0.73     17685

